In [ ]:
%%writefile mohanhathiv1.py



# Imports helper functions

from kaggle_environments.envs.halite.helpers import *



# Returns best direction to move from one position (fromPos) to another (toPos)

# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?

def getDirTo(fromPos, toPos, size):

    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)

    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)

    if fromY < toY: return ShipAction.NORTH

    if fromY > toY: return ShipAction.SOUTH

    if fromX < toX: return ShipAction.EAST

    if fromX > toX: return ShipAction.WEST



# Directions a ship can move

directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]



# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard

ship_states = {}



# Returns the commands we send to our ships and shipyards

def agent(obs, config):

    size = config.size

    board = Board(obs, config)

    me = board.current_player



    # If there are no ships, use first shipyard to spawn a ship.

    if len(me.ships) == 0 and len(me.shipyards) > 0:

        me.shipyards[0].next_action = ShipyardAction.SPAWN



    # If there are no shipyards, convert first ship into shipyard.

    if len(me.shipyards) == 0 and len(me.ships) > 0:

        me.ships[0].next_action = ShipAction.CONVERT

    

    # Turn the board into a 2d array showing halite at each location

    def get_map(obs):

        game_map = []

        for x in range(config.size):

            game_map.append([])

            for y in range(config.size):

                game_map[x].append(obs.halite[config.size * y + x]) # append amount of halite

        return game_map



    for ship in me.ships:

        if ship.next_action == None:

            

            ### Part 1: Set the ship's state 

            if board.step > 390:

                ship_states[ship.id] = "DEPOSIT"

            if ship.halite < 200: # If cargo is too low, collect halite

                ship_states[ship.id] = "COLLECT"

            if ship.halite > 5000: # If cargo gets very big, deposit halite

                ship_states[ship.id] = "DEPOSIT"

                

            ### Part 2: Use the ship's state to select an action

            if ship_states[ship.id] == "COLLECT":

                # If halite at current location running low, 

                # move to the adjacent square containing the most halite

                if ship.cell.halite < 200:

                    position = ship.position

                    game_map = get_map(obs)



                    # Go in direction with the most halite (look farther)

                    try:

                        possiblecells = [ship.cell.north.halite + game_map[position[0]][position[1]+2], ship.cell.east.halite + game_map[position[0]+2][position[1]], ship.cell.south.halite + game_map[position[0]][position[1]-2], ship.cell.west.halite + game_map[position[0]-2][position[1]]]

                    except IndexError:

                        possiblecells = [ship.cell.north.halite, ship.cell.east.halite, ship.cell.south.halite, ship.cell.west.halite]

                    

                    best = max(range(len(possiblecells)), key=possiblecells.__getitem__)

                    ship.next_action = directions[best]

            if ship_states[ship.id] == "DEPOSIT":

                # Move towards shipyard to deposit cargo

                direction = getDirTo(ship.position, me.shipyards[0].position, size)

                if direction: ship.next_action = direction

                

    return me.next_actions

In [ ]:
# Halite Game - Mohan
# 1. I've added logic to more smartly choose the next move (to optimize/go towards regions with the most halite)
# 2. I also added code to avoid collisions with other ships/shipyards
# 3. I made the ship automatically deposit all of its remaining halite at the end of the game
# 4. Finally, I tweaked some of the original parameters to make it perform better

%%writefile mohanhathiv2.py

# Imports helper functions
from kaggle_environments.envs.halite.helpers import *

# Returns best direction to move from one position (fromPos) to another (toPos)
# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
def getDirTo(fromPos, toPos, size):
    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)
    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)
    if fromY < toY: return ShipAction.NORTH
    if fromY > toY: return ShipAction.SOUTH
    if fromX < toX: return ShipAction.EAST
    if fromX > toX: return ShipAction.WEST

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]
directions1 = ['north', 'east', 'south', 'west']

# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_states = {}

# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player

    # If there are no ships, use first shipyard to spawn a ship.
    if len(me.ships) == 0 and len(me.shipyards) > 0:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    # If there are no shipyards, convert first ship into shipyard.
    if len(me.shipyards) == 0 and len(me.ships) > 0:
        me.ships[0].next_action = ShipAction.CONVERT
        
    # Turn the board into a 2d array showing halite at each location
    def get_map(obs):
        game_map = []
        for x in range(config.size):
            game_map.append([])
            for y in range(config.size):
                game_map[x].append(obs.halite[config.size * y + x]) # append amount of halite
        return game_map

    for ship in me.ships:
        if ship.next_action == None:
            
            ### Part 1: Set the ship's state 
            if board.step > 390:
                ship_states[ship.id] = "DEPOSIT"
            if ship.halite < 200: # If cargo is too low, collect halite
                ship_states[ship.id] = "COLLECT"
            if ship.halite > 5000: # If cargo gets very big, deposit halite
                ship_states[ship.id] = "DEPOSIT"
                
            ### Part 2: Use the ship's state to select an action
            if ship_states[ship.id] == "COLLECT":
                # If halite at current location running low, 
                # move to the adjacent square containing the most halite
                if ship.cell.halite < 200:
                    position = ship.position
                    game_map = get_map(obs)

                    # Go in direction with the most halite (look farther)
                    try:
                        possiblecells = [ship.cell.north.halite + game_map[position[0]][position[1]+2], ship.cell.east.halite + game_map[position[0]+2][position[1]], ship.cell.south.halite + game_map[position[0]][position[1]-2], ship.cell.west.halite + game_map[position[0]-2][position[1]]]
                    except IndexError:
                        possiblecells = [ship.cell.north.halite, ship.cell.east.halite, ship.cell.south.halite, ship.cell.west.halite]
                    
                    best = max(range(len(possiblecells)), key=possiblecells.__getitem__)
                    if directions1[best] == 'north':
                        if ship.cell.north.ship == None:  
                            ship.next_action = directions[best]
                    if directions1[best] == 'east':
                        if ship.cell.east.ship == None:  
                            ship.next_action = directions[best]
                    if directions1[best] == 'south':
                        if ship.cell.south.ship == None:  
                            ship.next_action = directions[best]
                    if directions1[best] == 'west':
                        if ship.cell.west.ship == None:  
                            ship.next_action = directions[best]
                    
            if ship_states[ship.id] == "DEPOSIT":
                # Move towards shipyard to deposit cargo
                direction = getDirTo(ship.position, me.shipyards[0].position, size)
                if direction:
                    if str(direction).lower() == 'north':
                        if ship.cell.north.ship == None:  
                            ship.next_action = direction
                    if str(direction).lower() == 'east':
                        if ship.cell.east.ship == None:  
                            ship.next_action = direction
                    if str(direction).lower() == 'south':
                        if ship.cell.south.ship == None:  
                            ship.next_action = direction
                    if str(direction).lower() == 'west':
                        if ship.cell.west.ship == None:  
                            ship.next_action = direction
                
    return me.next_actions

In [ ]:
from kaggle_environments import make, evaluate
env = make("halite", debug=True)
env.run(["mohanhathiv2.py", "mohanhathiv1.py", "random", "random"])
env.render(mode="ipython", width=800, height=600)